In [94]:
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

sys.path.append('../Scripts')
from Data_Processing import DataProcessing

from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import backend as K
from datetime import datetime
from sklearn.preprocessing import PowerTransformer

import joblib

In [95]:
ColumnTransformer = joblib.load('../Models/Column_Transformer.pkl')

In [96]:
df = DataProcessing('../Data/test.csv')

X = df.drop(columns=['Lap_Time'])

num_columns = X.select_dtypes(include='number').columns
PT = PowerTransformer(method='yeo-johnson')
X[num_columns] = PT.fit_transform(X[num_columns])

C:\Users\mcand\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [97]:
trans_X = ColumnTransformer.transform(X)

trans_X = trans_X.toarray()

In [98]:
def root_mean_squared_log_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y_true))))

In [99]:
model = load_model('../Models/NN_model.h5', custom_objects={'root_mean_squared_log_error': root_mean_squared_log_error})

In [100]:
y_predicted = model.predict(trans_X)

In [101]:
submission = pd.DataFrame(y_predicted, columns=['LAP_TIME'])

In [102]:
today = datetime.today().strftime('%m-%d-%y %H-%M')
submission.to_csv(f'../Submissions/Dare_In_Reality {today}.csv', index=False)

In [103]:
submission

,LAP_TIME
0,86.827858
1,85.247375
2,59.891621
3,95.622826
4,93.682396
...,...
415,43.369125
416,61.157917
417,45.386353
418,61.851620
